In [160]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
from sklearn import linear_model

In [161]:
in_file_name = 'data/grouped_paces_ju.tsv'
df_all = pd.read_csv(in_file_name, delimiter="\t")

In [162]:
#df = df_all[np.isfinite(df_all.pace_1)]
df = df_all

In [163]:
paces = df.as_matrix(["pace_2", "pace_3", "pace_4", "pace_5", "pace_6"])
logs = np.log(paces)
means = np.nanmean(logs, axis=1)
stdevs = np.nanstd(logs, axis=1)
df = df.assign(log_means=pd.Series(means).values)
df = df.assign(log_stdevs=pd.Series(stdevs).values)

/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [164]:
# Estimate values for all, but only use them if not other value is available
with_history = df[np.isfinite(df_all.pace_3)]
x = with_history.as_matrix(["mean_team_id"])
x = x.reshape(len(x), 1)

log_means = with_history.log_means.values.reshape(len(with_history.log_means), 1)
log_means_model = linear_model.LinearRegression()
log_means_model.fit(x, log_means)
estimated_log_means = log_means_model.predict(df.mean_team_id.values.reshape(len(df.mean_team_id), 1))
df = df.assign(estimated_log_means=estimated_log_means)

log_stdevs = with_history.log_stdevs.values.reshape(len(with_history.log_stdevs), 1)
log_stdevs_model = linear_model.LinearRegression()
log_stdevs_model.fit(x, log_stdevs)
estimated_log_stdevs = log_stdevs_model.predict(df.mean_team_id.values.reshape(len(df.mean_team_id), 1))
df = df.assign(estimated_log_stdevs=estimated_log_stdevs)



/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [165]:
df

,mean_team_id,teams,name,num_runs,num_valid_times,mean_pace,stdev,pace_1,pace_2,pace_3,pace_4,pace_5,pace_6,log_means,log_stdevs,estimated_log_means,estimated_log_stdevs
0,52.7,BROMMA-VÄLLINGBY SOK/JÄRFÄL;IFK GÖTEBORG,max peter bejmer,3,3,6.337,1.050,5.582,5.608,7.822,NaN,NaN,NaN,1.890567,0.166373,1.944796,0.087202
1,91.0,IFK GÖTEBORG;MARKBYGDENS OK,johan högstrand,4,4,6.198,0.468,5.591,5.902,6.571,6.728,NaN,NaN,1.854745,0.057003,1.962559,0.087890
2,13.2,IFK GÖTEBORG,vetle ruud bråten,4,4,5.750,0.181,5.688,5.636,5.615,6.060,NaN,NaN,1.752109,0.035106,1.926477,0.086493
3,33.2,IFK GÖTEBORG,jonas pilblad,5,5,6.083,0.389,5.978,5.772,6.819,5.756,6.089,NaN,1.807365,0.068628,1.935752,0.086852
4,17.5,IFK GÖTEBORG;OK KÅRE,fredrik edn,2,2,6.333,0.472,5.861,6.805,NaN,NaN,NaN,NaN,1.917658,0.000000,1.928471,0.086570
5,10.5,IFK LIDINGÖ SOK;IFK GÖTEBORG,fredrik bakkman,2,2,5.483,0.028,5.511,5.455,NaN,NaN,NaN,NaN,1.696533,0.000000,1.925225,0.086444
6,16.8,IFK GÖTEBORG,eskil kinneberg,4,4,5.623,0.084,5.611,5.494,5.719,5.668,NaN,NaN,1.727429,0.017203,1.928146,0.086558
7,21.3,KOOVEE,topi anjala,6,6,6.080,0.516,5.621,6.100,5.977,7.164,5.642,5.974,1.816586,0.080558,1.930233,0.086638
8,1.0,KOOVEE,olexander kratov,1,1,5.874,0.000,5.874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.920819,0.086274
9,6.7,VAAJAKOSKEN TERÄ;KOOVEE,jani lakanen,6,6,6.044,0.456,5.874,5.734,6.208,6.717,5.326,6.403,1.801252,0.082382,1.923462,0.086376


In [166]:
# s = sigma and scale = exp(mu).
df['final_log_means'] = np.where(np.isfinite(df["log_means"]) & df["log_means"]>0, df["log_means"], df["estimated_log_means"])
final_log_means = np.exp(df['final_log_means']) 
df['final_log_stdevs'] = np.where(np.isfinite(df["log_stdevs"]) & df["log_stdevs"]>0, df["log_stdevs"], df["estimated_log_stdevs"])
final_log_stdevs = df['final_log_stdevs']

intervals95 = lognorm.interval(0.95, s = final_log_stdevs, scale = final_log_means)
means = lognorm.mean(s = final_log_stdevs, scale = final_log_means)
medians = lognorm.median(s = final_log_stdevs, scale = final_log_means)

In [167]:

df = df.assign(interval95_start = pd.Series( intervals95[0] ).values)
df = df.assign(interval95_end = pd.Series( intervals95[1] ).values)
df = df.assign(p1_in_interval = pd.Series( (df.interval95_start <= df.pace_1) & (df.interval95_end >= df.pace_1) ).values)
df = df.assign(mean_ln = pd.Series(means).values)
df = df.assign(med_ln = pd.Series(medians).values)

In [168]:
df

,mean_team_id,teams,name,num_runs,num_valid_times,mean_pace,stdev,pace_1,pace_2,pace_3,...,log_stdevs,estimated_log_means,estimated_log_stdevs,final_log_means,final_log_stdevs,interval95_start,interval95_end,p1_in_interval,mean_ln,med_ln
0,52.7,BROMMA-VÄLLINGBY SOK/JÄRFÄL;IFK GÖTEBORG,max peter bejmer,3,3,6.337,1.050,5.582,5.608,7.822,...,0.166373,1.944796,0.087202,1.890567,0.166373,4.780198,9.176560,True,6.715426,6.623124
1,91.0,IFK GÖTEBORG;MARKBYGDENS OK,johan högstrand,4,4,6.198,0.468,5.591,5.902,6.571,...,0.057003,1.962559,0.087890,1.854745,0.057003,5.714579,7.145405,False,6.400459,6.390069
2,13.2,IFK GÖTEBORG,vetle ruud bråten,4,4,5.750,0.181,5.688,5.636,5.615,...,0.035106,1.926477,0.086493,1.752109,0.035106,5.383299,6.177514,True,5.770305,5.766750
3,33.2,IFK GÖTEBORG,jonas pilblad,5,5,6.083,0.389,5.978,5.772,6.819,...,0.068628,1.935752,0.086852,1.807365,0.068628,5.327360,6.971798,True,6.108733,6.094365
4,17.5,IFK GÖTEBORG;OK KÅRE,fredrik edn,2,2,6.333,0.472,5.861,6.805,NaN,...,0.000000,1.928471,0.086570,1.917658,0.086570,5.743009,8.063374,True,6.830548,6.805000
5,10.5,IFK LIDINGÖ SOK;IFK GÖTEBORG,fredrik bakkman,2,2,5.483,0.028,5.511,5.455,NaN,...,0.000000,1.925225,0.086444,1.696533,0.086444,4.604825,6.462140,True,5.475420,5.455000
6,16.8,IFK GÖTEBORG,eskil kinneberg,4,4,5.623,0.084,5.611,5.494,5.719,...,0.017203,1.928146,0.086558,1.727429,0.017203,5.439636,5.819101,True,5.627003,5.626170
7,21.3,KOOVEE,topi anjala,6,6,6.080,0.516,5.621,6.100,5.977,...,0.080558,1.930233,0.086638,1.816586,0.080558,5.252450,7.202858,True,6.170816,6.150825
8,1.0,KOOVEE,olexander kratov,1,1,5.874,0.000,5.874,NaN,NaN,...,NaN,1.920819,0.086274,1.920819,0.086274,5.764539,8.084205,True,6.851998,6.826545
9,6.7,VAAJAKOSKEN TERÄ;KOOVEE,jani lakanen,6,6,6.044,0.456,5.874,5.734,6.208,...,0.082382,1.923462,0.086376,1.801252,0.082382,5.154065,7.118654,True,6.077817,6.057227


In [169]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
mean_err = rmse(df.mean_ln, df.pace_1)
med_err = rmse(df.med_ln, df.pace_1)
(mean_err, med_err)

(2.541390016173329, 2.5301453673082346)

In [170]:
np.mean(df.p1_in_interval)

0.6590752242926156

In [171]:
df.to_csv('data/log_normal_estimates_ju.tsv', sep="\t")

In [172]:
runs17 = pd.read_csv('data/csv-results_j2017_ju.tsv', delimiter="\t")


In [173]:
def get_estimate_row(row):
    name = row["competitor-name"].lower()
    
    by_name = df[df['name'] == name]
    by_name_and_colon = df[df['name'].str.contains(name +":", regex=False)]

    runners = by_name.append(by_name_and_colon)
    if(len(runners) == 1):
        return runners
    team_name = row["team-name"].upper()
    runners = runners[runners['teams'].str.contains(team_name, regex=False)]
    if(len(runners) == 1):
        return runners
    print(f"name '{name}' team_name '{team_name}'")
    print(f"by_name {len(by_name)} by_name_and_colon {len(by_name_and_colon)} runners {len(runners)}")
    print(f"Duplicate runner {runners}")
    #print(f"TEAMS by_name_and_colon {by_name_and_colon['teams']}")
    return runners.sort_values("num_runs", ascending = False).head(1)

def get_estimate_params(row):
    estimate_row = get_estimate_row(row)
    #print(f"estimate_row final_log_means {estimate_row.final_log_means} {estimate_row.final_log_stdevs}")
    final_log_means = estimate_row.final_log_means.values[0]
    final_log_stdevs = estimate_row.final_log_stdevs.values[0]
    return pd.Series({"final_log_means": final_log_means, "final_log_stdevs": final_log_stdevs})

#runs17 = runs17[runs17['team-name'].str.contains("Reak")]
estimate_params = runs17.apply(lambda row: get_estimate_params(row), axis=1)
runs17 = runs17.assign(final_log_means = estimate_params.final_log_means)
runs17 = runs17.assign(final_log_stdevs = estimate_params.final_log_stdevs)

name 'panu kärkkäinen' team_name 'JÄMSÄN RETKI-VEIKOT'
by_name 0 by_name_and_colon 4 runners 2
Duplicate runner       mean_team_id                        teams  \
1841         570.0          JÄMSÄN RETKI-VEIKOT   
1843         954.0  JÄMSÄN RETKI-VEIKOT/JUKOLA2   

                                             name  num_runs  num_valid_times  \
1841          panu kärkkäinen:JÄMSÄN RETKI-VEIKOT         4                3   
1843  panu kärkkäinen:JÄMSÄN RETKI-VEIKOT/JUKOLA2         1                1   

      mean_pace  stdev  pace_1  pace_2  pace_3    ...      log_stdevs  \
1841      9.109  1.401   7.938    8.31  11.079    ...        0.143796   
1843      8.686  0.000   8.686     NaN     NaN    ...             NaN   

      estimated_log_means  estimated_log_stdevs  final_log_means  \
1841             2.184710              0.096495         2.261256   
1843             2.362802              0.103393         2.362802   

      final_log_stdevs  interval95_start  interval95_end  p1_in_inte

In [174]:
def leg_dist(leg):
    dist = [12.7, 14.2, 12.3, 7.6, 7.9, 10.9, 13.8]
    return dist[leg - 1]

runs17 = runs17.assign(leg_dist = runs17["leg-nro"].apply(lambda nro: leg_dist(nro)))

final_means = np.exp(runs17.final_log_means)
#intervals95 = lognorm.interval(0.95, s = runs17.final_log_stdevs, scale = final_means)

#runs17 = runs17.assign(start95 = intervals95[0] * runs17["leg_dist"])
#runs17 = runs17.assign(end95 = intervals95[1] * runs17["leg_dist"])

runs17["est_median"] = lognorm.median(s = runs17.final_log_stdevs, scale = final_means) * runs17["leg_dist"]


In [175]:
runs17

,team-id,placement,team-time,team-name,team-nro,leg-nro,emit,leg-time,competitor-name,control-times,final_log_means,final_log_stdevs,leg_dist,est_median
0,5,1.0,27105.0,IFK Göteborg,1,1,1217511.0,4287.0,Max Peter Bejmer,927;1080;1144;1193;1334;1436;1548;1612;1725;19...,1.890567,0.166373,12.7,84.113679
1,5,1.0,27105.0,IFK Göteborg,1,2,1217440.0,4797.0,Johan Högstrand,905;981;1103;1148;1314;1380;1451;1542;1605;167...,1.854745,0.057003,14.2,90.738981
2,5,1.0,27105.0,IFK Göteborg,1,3,1221058.0,4198.0,Vetle Ruud Bråten,897;1025;1118;1165;1293;1386;1475;1537;1616;18...,1.752109,0.035106,12.3,70.931025
3,5,1.0,27105.0,IFK Göteborg,1,4,1217402.0,2762.0,Jonas Pilblad,325;457;659;917;989;1107;1574;1809;1897;1971;2...,1.807365,0.068628,7.6,46.317172
4,5,1.0,27105.0,IFK Göteborg,1,5,1217509.0,2743.0,Fredrik Edn,285;442;603;802;926;979;1074;1531;1711;1799;18...,1.917658,0.086570,7.9,53.759500
5,5,1.0,27105.0,IFK Göteborg,1,6,1216571.0,3670.0,Fredrik Bakkman,260;390;642;1101;1322;1404;1591;1729;1807;1988...,1.696533,0.086444,10.9,59.459500
6,5,1.0,27105.0,IFK Göteborg,1,7,1217441.0,4646.0,Eskil Kinneberg,304;524;717;1131;1189;1309;1411;1862;2154;2231...,1.727429,0.017203,13.8,77.641148
7,1,2.0,27171.0,Koovee,1,1,1201850.0,4317.0,Topi Anjala,933;1087;1150;1198;1338;1438;1537;1605;1720;19...,1.816586,0.080558,12.7,78.115482
8,1,2.0,27171.0,Koovee,1,2,1230611.0,5040.0,Olexander Kratov,896;1036;1151;1197;1375;1441;1508;1599;1666;17...,1.920819,0.086274,14.2,96.936939
9,1,2.0,27171.0,Koovee,1,3,1214773.0,4335.0,Jani Lakanen,949;1096;1193;1241;1375;1495;1598;1660;1754;20...,1.801252,0.082382,12.3,74.503898


In [176]:
by_teams = runs17.set_index(["team-id", "leg-nro"]).unstack()


In [177]:
for i in range(1,8):
    by_teams["fin_real", i] = np.sum([ by_teams["leg-time"][j] / 60 for j in range(1,i+1) ], axis=0)
by_teams.fin_real

leg-nro,1,2,3,4,5,6,7
team-id,,,,,,,
1,71.950000,155.950000,228.200000,273.250000,317.416667,375.933333,452.800000
2,77.383333,170.700000,241.633333,296.166667,341.000000,405.933333,486.383333
3,83.800000,173.333333,245.283333,292.850000,346.200000,411.166667,488.283333
4,71.700000,162.183333,232.483333,278.366667,323.850000,385.600000,463.966667
5,71.450000,151.400000,221.366667,267.400000,313.116667,374.283333,451.716667
6,78.083333,163.700000,232.133333,279.116667,322.166667,385.700000,462.450000
7,77.333333,163.950000,229.083333,276.766667,321.766667,388.150000,470.616667
8,78.850000,172.183333,239.666667,285.333333,332.750000,395.133333,473.833333
9,72.183333,156.000000,231.633333,278.933333,324.266667,392.650000,470.483333


In [178]:
for i in range(1,8):
    by_teams["fin_med", i] = np.sum([ by_teams["est_median"][j] for j in range(1,i+1) ], axis=0)
by_teams.fin_med

leg-nro,1,2,3,4,5,6,7
team-id,,,,,,,
1,78.115482,175.052421,249.556318,294.800274,343.227568,405.495225,484.482567
2,79.403068,170.836523,247.099961,296.512012,340.698314,415.142172,493.400602
3,82.681744,171.086014,248.612066,300.541954,355.747154,421.257737,501.327056
4,74.788233,163.609640,243.903681,290.084376,341.344467,407.003929,483.900146
5,84.113679,174.852660,245.783685,292.100857,345.860357,405.319857,482.961005
6,78.010408,166.852545,240.740997,294.185775,342.515740,409.398963,488.752784
7,84.863935,170.276536,252.927409,298.837625,348.035640,415.843943,501.567728
8,81.560488,168.658645,240.538165,288.596949,335.608397,398.441582,482.657413
9,75.698732,158.282669,243.415769,298.035707,347.314457,416.714352,497.798274


In [179]:

def get_simulated_medians(row):
    samples = pd.DataFrame()
    for i in range(1,8):
        samples[i] = row["leg_dist"][i] * lognorm.rvs(s = row["final_log_stdevs"][i], scale = np.exp(row["final_log_means"][i]), size = 10000)

    samples_sums = pd.DataFrame()
    for i in range(1,8):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    """
    for i in range(1,8):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["competitor-name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1,8):
        plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1,8)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1,8)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1,8)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1,8)}
    new_cols = {**fin_start95_dict, **fin_end95_dict, **fin_median_dict, **fin_mean_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

simulated = by_teams.apply(get_simulated_medians, axis=1)
simulated



,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_start95_5,fin_start95_6,fin_start95_7,fin_end95_1,fin_end95_2,fin_end95_3,...,fin_median_5,fin_median_6,fin_median_7,fin_mean_1,fin_mean_2,fin_mean_3,fin_mean_4,fin_mean_5,fin_mean_6,fin_mean_7
team-id,,,,,,,,,,,,,,,,,,,,,
1,66.793384,155.674151,226.827532,270.889521,318.943149,380.492594,456.279145,91.423610,197.501255,275.685703,...,344.135182,406.397754,485.776999,78.396244,175.619059,250.460221,295.775252,344.270531,406.703310,486.048160
2,71.538728,159.079221,231.669897,276.124641,320.135109,391.171359,466.608688,88.504553,184.248118,264.753958,...,341.589609,416.419257,494.817617,79.594125,171.182752,247.661156,297.581041,341.812173,416.624856,495.027413
3,64.185422,150.269616,226.459457,276.738284,330.225456,394.157277,472.750544,106.355805,196.819523,275.647868,...,356.676141,422.359462,502.568765,83.313909,171.923681,249.603960,301.714867,357.020306,422.764404,503.050465
4,72.277780,148.447115,224.234520,268.251529,317.393733,381.995116,458.680140,77.440769,180.812860,265.935374,...,342.422671,408.163297,485.199084,74.810341,163.941607,244.465528,291.001934,342.557950,408.408370,485.406109
5,61.013161,149.610976,219.965261,265.824904,317.854556,376.137624,453.325710,116.038401,208.844662,279.724797,...,346.139110,406.033810,483.791559,85.036844,175.894428,246.839891,293.283978,347.256072,407.037088,484.692914
6,77.090895,158.133452,224.951960,276.385628,323.500200,389.714366,465.753883,78.968869,176.496219,259.506481,...,343.068967,409.965151,489.730270,78.017949,167.006282,241.296544,294.952366,343.370422,410.295757,490.027239
7,60.871923,143.050006,222.023081,267.712278,316.609571,380.572051,464.296366,117.906528,206.375927,292.946366,...,349.199871,417.235047,503.271000,85.907216,171.583105,254.774610,300.715793,349.963659,418.166957,504.096682
8,65.508400,147.871706,216.651533,263.863681,310.565340,371.902789,453.286031,102.159841,192.829116,268.935791,...,336.671276,399.382231,483.911005,82.121878,169.429262,241.626781,289.875275,336.963951,399.882832,484.301337
9,59.623539,140.057313,222.052415,273.790572,320.100919,388.259695,467.374347,95.788393,180.193459,268.682181,...,348.308001,417.875619,499.419182,76.237019,158.908848,244.270630,299.121475,348.791052,418.302689,499.609684


In [180]:
by_teams_2 = by_teams.copy()
by_teams_2.columns = [f'{x[0]}_{x[1]}' for x in by_teams_2.columns]
by_teams_2.reset_index()
by_teams_2

,placement_1,placement_2,placement_3,placement_4,placement_5,placement_6,placement_7,team-time_1,team-time_2,team-time_3,...,fin_real_5,fin_real_6,fin_real_7,fin_med_1,fin_med_2,fin_med_3,fin_med_4,fin_med_5,fin_med_6,fin_med_7
team-id,,,,,,,,,,,,,,,,,,,,,
1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,27171.0,27171.0,27171.0,...,317.416667,375.933333,452.800000,78.115482,175.052421,249.556318,294.800274,343.227568,405.495225,484.482567
2,21.0,21.0,21.0,21.0,21.0,21.0,21.0,29186.0,29186.0,29186.0,...,341.000000,405.933333,486.383333,79.403068,170.836523,247.099961,296.512012,340.698314,415.142172,493.400602
3,24.0,24.0,24.0,24.0,24.0,24.0,24.0,29299.0,29299.0,29299.0,...,346.200000,411.166667,488.283333,82.681744,171.086014,248.612066,300.541954,355.747154,421.257737,501.327056
4,5.0,5.0,5.0,5.0,5.0,5.0,5.0,27842.0,27842.0,27842.0,...,323.850000,385.600000,463.966667,74.788233,163.609640,243.903681,290.084376,341.344467,407.003929,483.900146
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27105.0,27105.0,27105.0,...,313.116667,374.283333,451.716667,84.113679,174.852660,245.783685,292.100857,345.860357,405.319857,482.961005
6,4.0,4.0,4.0,4.0,4.0,4.0,4.0,27750.0,27750.0,27750.0,...,322.166667,385.700000,462.450000,78.010408,166.852545,240.740997,294.185775,342.515740,409.398963,488.752784
7,9.0,9.0,9.0,9.0,9.0,9.0,9.0,28240.0,28240.0,28240.0,...,321.766667,388.150000,470.616667,84.863935,170.276536,252.927409,298.837625,348.035640,415.843943,501.567728
8,11.0,11.0,11.0,11.0,11.0,11.0,11.0,28433.0,28433.0,28433.0,...,332.750000,395.133333,473.833333,81.560488,168.658645,240.538165,288.596949,335.608397,398.441582,482.657413
9,8.0,8.0,8.0,8.0,8.0,8.0,8.0,28232.0,28232.0,28232.0,...,324.266667,392.650000,470.483333,75.698732,158.282669,243.415769,298.035707,347.314457,416.714352,497.798274


In [181]:
result = pd.concat([by_teams_2, simulated], axis=1, join='inner')
result

,placement_1,placement_2,placement_3,placement_4,placement_5,placement_6,placement_7,team-time_1,team-time_2,team-time_3,...,fin_median_5,fin_median_6,fin_median_7,fin_mean_1,fin_mean_2,fin_mean_3,fin_mean_4,fin_mean_5,fin_mean_6,fin_mean_7
team-id,,,,,,,,,,,,,,,,,,,,,
1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,27171.0,27171.0,27171.0,...,344.135182,406.397754,485.776999,78.396244,175.619059,250.460221,295.775252,344.270531,406.703310,486.048160
2,21.0,21.0,21.0,21.0,21.0,21.0,21.0,29186.0,29186.0,29186.0,...,341.589609,416.419257,494.817617,79.594125,171.182752,247.661156,297.581041,341.812173,416.624856,495.027413
3,24.0,24.0,24.0,24.0,24.0,24.0,24.0,29299.0,29299.0,29299.0,...,356.676141,422.359462,502.568765,83.313909,171.923681,249.603960,301.714867,357.020306,422.764404,503.050465
4,5.0,5.0,5.0,5.0,5.0,5.0,5.0,27842.0,27842.0,27842.0,...,342.422671,408.163297,485.199084,74.810341,163.941607,244.465528,291.001934,342.557950,408.408370,485.406109
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27105.0,27105.0,27105.0,...,346.139110,406.033810,483.791559,85.036844,175.894428,246.839891,293.283978,347.256072,407.037088,484.692914
6,4.0,4.0,4.0,4.0,4.0,4.0,4.0,27750.0,27750.0,27750.0,...,343.068967,409.965151,489.730270,78.017949,167.006282,241.296544,294.952366,343.370422,410.295757,490.027239
7,9.0,9.0,9.0,9.0,9.0,9.0,9.0,28240.0,28240.0,28240.0,...,349.199871,417.235047,503.271000,85.907216,171.583105,254.774610,300.715793,349.963659,418.166957,504.096682
8,11.0,11.0,11.0,11.0,11.0,11.0,11.0,28433.0,28433.0,28433.0,...,336.671276,399.382231,483.911005,82.121878,169.429262,241.626781,289.875275,336.963951,399.882832,484.301337
9,8.0,8.0,8.0,8.0,8.0,8.0,8.0,28232.0,28232.0,28232.0,...,348.308001,417.875619,499.419182,76.237019,158.908848,244.270630,299.121475,348.791052,418.302689,499.609684


In [182]:
result.fin_real_1
(rmse(result["fin_mean_1"], result["fin_real_1"]), rmse(result["fin_median_1"], result["fin_real_1"]))

(20.4759983552587, 20.004045146218314)

In [183]:
[ (rmse(result[f"fin_mean_{leg}"], result[f"fin_real_{leg}"]), rmse(result[f"fin_median_{leg}"], result[f"fin_real_{leg}"])) for leg in range(1,8)]

[(20.4759983552587, 20.004045146218314),
 (33.664512215359274, 33.10186470781429),
 (45.967819114288936, 45.364523028774045),
 (49.61146523227387, 49.14572537332332),
 (56.03860194522178, 55.74244316513179),
 (64.36717304178279, 64.22714629625366),
 (73.69417709523377, 73.43495306189679)]

In [184]:
result.to_csv('data/team_estimates_ju2017.tsv', sep="\t")

### 